# Last Cycle
Now that the cycle stabilized we can cycle one more time to track relevant quantities:
- Mutual Information
- Work
- Heat
- Internal Energy (?)

In [ ]:
using MKL
using Revise
using Dates
using CSV
using DataFrames

CSV_FILE = "./simulations/simulations_ledger.csv";

csv = CSV.read(CSV_FILE, DataFrame, header=[1,2]);
dirs = ["simulations/simulation_$n" for n in csv[!, "meta_name"]];

includet("./src/main.jl")

cycle_in_dir(dirs[1])

"./simulations/simulations_ledger.csv"

In [ ]:
csv = CSV.read(CSV_FILE, DataFrame, header=[1,2]);
println("There are $(nrow(csv)) simulation files")

There are 900 simulation files


In [ ]:
ρᵢ

In [ ]:
includet("./modules/OpticalCavity.jl")
includet("./modules/Thermodynamics.jl")
includet("./modules/Phaseonium.jl")
includet("./modules/BosonicOperators.jl")
includet("./modules/Measurements.jl")

includet("./src/main.jl")

# Run a fast cycle to compile the code
config, ndims = init(".")
state = load_or_create('.', config)
state.ρ₁_evolution = [] 
state.ρ₂_evolution = []
state.c₁_evolution = []
state.c₂_evolution = []
# Cycle
isochore_time = config["stroke_time"]["isochore"]
isochore_samplings = config["samplings"]["isochore"]
adiabatic_time = config["stroke_time"]["adiabatic"]
adiabatic_samplings = config["samplings"]["adiabatic"]
system_evolution = []
cycle_steps = []
state, system_evolution = Main.cycle(
    state, Δt, system_evolution, cycle_steps, 
    isochore_time, isochore_samplings, adiabatic_time, adiabatic_samplings, stdout);

Fast Simulation - just to compile the code

Initial Temperature of the Cavity: 2.0
Apparent Temperature carried by Hot Phaseonium atoms: 2.4999999999999996
Apparent Temperature carried by Cold Phaseonium atoms: 1.0
Starting with a new cascade system (contracted)
Initial Temperature of the Cavities: 2.0 - 2.0
Isochoric Stroke
Adiabatic Expansion
f1:2.5 - f2:2.5
a1:0.31086011877150366 - a2:0.29383336078710576
f1:2.5 - f2:2.5
a1:0.31077288688012183 - a2:0.2937513987761782
f1:2.5 - f2:2.5
a1:0.3106856835265548 - a2:0.29366946323328413
f1:2.5 - f2:2.5
a1:0.31059850869876815 - a2:0.29358755414745996
f1:2.5 - f2:2.5
a1:0.31051136238505395 - a2:0.29350567150805773
f1:2.5 - f2:2.5
a1:0.3104242445739378 - a2:0.29342381530466977
f1:2.5 - f2:2.5
a1:0.31033715525372463 - a2:0.29334198552666324
f1:2.5 - f2:2.5
a1:0.310250094412976 - a2:0.293260182163662
f1:2.5 - f2:2.5
a1:0.3101630620390825 - a2:0.29317840520411886
f1:2.5 - f2:2.5
a1:0.31007605812091743 - a2:0.29309665463797074
f1:2.5 - f2:2.5
a1:0.

In [ ]:
# Take all directories in the simulation csv
dirs = ["simulations/simulation_$n" for n in csv[!, "meta_name"]];
length(dirs)

900

In [6]:
for dir in dirs[1:2]
    println(stdout, "Analyzing $dir")
    config, ndims = init(dir)

    total_cycles = config["loading"]["past_cycles"]
    system_evolution = []

    # Load or create state
    state = load_or_create(dir, config)
    # Reset state
    state.ρ₁_evolution = [] 
    state.ρ₂_evolution = []
    state.c₁_evolution = []
    state.c₂_evolution = []

    # Report file
    open(dir * "/report.txt", "w") do io
        println(io, "Starting simulation at time $(now()) with config:")
        print_config(io, config)
    end
    # Visualization folder
    mkpath(dir * "/visualization")

    # Cycle
    total_cycles = 0
    isochore_time = config["stroke_time"]["isochore"]
    isochore_samplings = config["samplings"]["isochore"]
    adiabatic_time = config["stroke_time"]["adiabatic"]
    adiabatic_samplings = config["samplings"]["adiabatic"]

    total_cycle_time = isochore_samplings * 2 + 2 * adiabatic_samplings
    cycle_steps = []  # This keeps track of the total time of each stroke

    open(dir * "/report.txt", "a") do io
        redirect_stdout(io)

        global state
        global system_evolution
        global total_cycles

        try
            for t in 1:config["meta"]["cycles"]
                println("Cycle $t")
                state, system_evolution = Main.cycle(
                    state, Δt, system_evolution, cycle_steps, 
                    isochore_time, isochore_samplings, adiabatic_time, adiabatic_samplings, io);
                total_cycles += 1
                flush(io)
            end
        catch e 
            println(io, "\n\nError '$e' at time $(now()) after $(length(system_evolution)) steps")
            println("\n\nError '$e' at time $(now()) after $(length(system_evolution)) steps")
            # error("Error $e: ending cycle")
        else
            println(io, "\n\nFinished at time $(now()) after $(length(system_evolution)) steps")
            println(io, "********************** ************")
            serialize(dir * "/state_$(total_cycles)C.jl", state)
            serialize(dir * "/state_evolution_$(total_cycles)C.jl", system_evolution)
            serialize(dir * "/cavity1_evolution_$(total_cycles)C.jl", state.c₁_evolution)
            serialize(dir * "/cavity2_evolution_$(total_cycles)C.jl", state.c₂_evolution)
        end
    end
    println("Cycle len: $(size(system_evolution)[1] / total_cycles) steps")
end

Analyzing simulations/simulation_69c38282-2

Initial Temperature of the Cavity: 2.0
Apparent Temperature carried by Hot Phaseonium atoms: 2.4999999999999996
Apparent Temperature carried by Cold Phaseonium atoms: 1.0
Starting with a new cascade system (contracted)


Initial Temperature of the Cavities: 2.0 - 2.0


In [ ]:
stdout

### Check stability of the Cycle

In [ ]:
println(size(system_evolution))
println(size(state.c₁_evolution))
state_evolution1 = state.ρ₁_evolution[end-total_cycle_time+1:end] # last cycle
cavity_evolution1 = state.c₁_evolution[end-total_cycle_time+1:end] # last cycle
state_evolution2 = state.ρ₂_evolution[end-total_cycle_time+1:end] # last cycle
cavity_evolution2 = state.c₂_evolution[end-total_cycle_time+1:end] # last cycle

g = measure_and_plot("Frequency", "Energy", state_evolution1, cavity_evolution1, "S1"; α=π, title="Cavities last cycle")
g = measure_and_plot("Frequency", "Energy", state_evolution2, cavity_evolution2, "S2"; α=π, g=g)

In [ ]:
total_cycles

In [ ]:
state_evolution2_last = state.ρ₂_evolution[end-total_cycle_time+1:end]
cavity_evolution2_last = state.c₂_evolution[end-total_cycle_time+1:end]
state_evolution2_last1 = state.ρ₂_evolution[end-2*total_cycle_time+1:end-total_cycle_time]
cavity_evolution2_last1 = state.c₂_evolution[end-2*total_cycle_time+1:end-total_cycle_time]
state_evolution2_last2 = state.ρ₂_evolution[end-3*total_cycle_time+1:end-2*total_cycle_time]
cavity_evolution2_last2 = state.c₂_evolution[end-3*total_cycle_time+1:end-2*total_cycle_time]

g = measure_and_plot("Frequency", "Energy", state_evolution2_last, cavity_evolution2_last, "last"; α=π)
g = measure_and_plot("Frequency", "Energy", state_evolution2_last1, cavity_evolution2_last1, "last-1"; α=π, g=g)
# g = measure_and_plot("Frequency", "Energy", state_evolution2_last2, cavity_evolution2_last2, "last-1"; α=π, g=g)

In [ ]:
savefig(g, dir * "/last_cycle.png")

## Save Last Cycle

In [ ]:
α0 = config["cavity1"]["alpha"]

last_evolution = system_evolution[end-total_cycle_time+1:end]
ω₁_evolution = [α0 / l1 for l1 in state.c₁_evolution[end-total_cycle_time+1:end]]
ω₂_evolution = [α0 / l2 for l2 in state.c₂_evolution[end-total_cycle_time+1:end]]

dict_evolution = Dict(string(i)=>last_evolution[i] for i in 1:length(last_evolution))
cycle_steps = Vector{Float64}(cycle_steps)

npzwrite(dir * "/cycle_steps.npy", cycle_steps[end-3:end])
npzwrite(dir * "/cavities_evolution_1cycle.npz", c1=ω₁_evolution, c2=ω₂_evolution)
npzwrite(dir * "/cascade_evolution_1cycle.npz", dict_evolution)